In [1]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime


In [2]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', '

In [6]:
# Get stock prices and returns
startdate = datetime.datetime(2020,3,23)
enddate = datetime.datetime(2020,3,27)

temp_tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 
                'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 
                'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'ABC', 'AME',
                'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AIZ', 'T',
                'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY',
                'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 
                'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 
                'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB',
                'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 
                'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 
                'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 
                'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 
                'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 
                'EPAM', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR',
                'XOM', 'FFIV','FDS', 'FAST']

file = open("stocks.csv",'w',newline='')
for ticker in temp_tickers:
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass

file.close()


In [7]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets.csv","w",newline='')
alltweets = csv.writer(file)

for ticker in temp_tickers:
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets,q=query,lang='en',tweet_mode='extended').items(10)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass

file.close()


 MMM  Waiting for BUY signal on    MMM    Register for 7 Day Trial Access at https   t co WBARPaK0Cc   https   t co C6OxCBuvh9
 ENB  MMM  MO  NVS  UGI   5  Yielding Dividend Aristocrats You ll Want To Own In The Coming Bear Market https   t co TL0TMYQ1g0 https   t co I192pEf9v7
 ENB  MMM  MO NEW ARTICLE   5  Yielding Dividend Aristocrats You ll Want To Own In The Coming Bear Market https   t co W8c9XMhzus Get all the latest  ENB related news here   https   t co Yuyi4PQvkZ
 FOR  A  RATE  I  CRY  TWO  AT  A  TIME  MMM 
3M Company  NYSE   MMM    Peering Into 3M Company s Recent Short Interest https   t co Fjt845zO4l
 MMM  NP  SWM NEW ARTICLE   A Georgia Wedding Between 2 Specialty Material Companies https   t co Wh6Dca5kyp Get all the latest  MMM related news here   https   t co vm9QziaCd9
A Georgia Wedding Between 2 Specialty Material Companies  NP  SWM  MMM  stocks  investing  valueinvesting   https   t co RJMWiMb4l3
 MMM 13 has crossed above    48 and ERROR retrieving 200 data  check m

 GTCH  Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS     GTCH  S  MNDT  GOOG  ACN https   t co YnJ2HqEoiM
 GTCH  Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS     GTCH  S  MNDT  GOOG  ACN https   t co v8GksQQf2z
In Case you Missed it   Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS   MNDT   NasdaqGS   GOOG   NYSE   ACN   SentinelOne  Mandiant  Google  Accenture  Stocks  markets  Finance  Business  Investing  tech  technology https   t co bQZCDGdPvU https   t co 599ubFmByD
Cybersecurity Solutions Stocks in the news    OTC PINK   GTCH   NYSE   S   NasdaqGS   MNDT   NasdaqGS   GOOG   NYSE   ACN  https   t co xf7sHGDmq4 https   t co 9xFBuuYIqT
In Case you Missed it   Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS   MNDT   NasdaqGS   GOOG   NYSE   ACN   SentinelOne  Mandiant  Google  Accenture  Stocks  markets  Finance  Business  Investing  te

INIO s Gold Daily Stock Screener   QRVO   CRAI   THRM   HSIC   CHUY   MKSI   ACLS   HI   RUTH   EVTC   MOH   AAP   NXPI   G
 AAP Best Room on the net   Stay Turned https   t co n5TZnOE5Cp
Tekmetric Is Now Advance Auto Parts  Recommended Shop Management System https   t co LOPcSpJ6eU   tekmetric  AdvanceAuto  AAP  aftermarket  autoparts  autocare https   t co IVcS9JmXjg
 AAP All that you need to know  Shorts haven t covered a single share  https   t co n5TZnODxMR
 AAP  BBY  BX  DLR  EPD  MMM  MS  QSR  TROW  UPS   My Top 10 High Yield Dividend Stocks For April 2022 https   t co 5GF3dk98nP https   t co 9yxkoS2ceG
 AAP  BBY  BX  DLR  EPD  MMM  MS  QSR  TROW  UPS   My Top 10 High Yield Dividend Stocks For April 2022 https   t co e8URFykIIl https   t co QAY5Sx4FFv
 AAP  BBY  BX NEW ARTICLE   My Top 10 High Yield Dividend Stocks For April 2022 https   t co tL5QM5WDo0 Get all the latest  AAP related news here   https   t co KrYJTjg54f
 AAP   When the Price of  Advance Auto Parts  AAP  Talks  P

Akamai Technologies  Inc  traded 7  above it s 50 DMA and 7  above it s 200 DMA  AKAM
New 52 Week High today   AZN  BREZ  IGF  STE  BMY  RACYU  VOLT  FXU  MUSA  VCXB  REGN  CEG  EXLS  AKAM  PST  THCA  RAAX  FTXG  TYO  TBX     https   t co 4Iw0QeR3p8
 AKAM Implied Volatility expiration 04 08 2022  maxpain  options https   t co 0TAPO3Z1oj https   t co auor370B8x
 NASDAQ  NewHigh Count Technology  TENB 2  AKAM 2  QLYS 2  CNXN 2  LLNW 2  EXLS 1  GVP 1  JKHY 1  AOSL 1  COHR 1  From 2022 03 29 
Akamai Technologies  Inc current quarter revenue growth YoY of 7 00  ranked 2639 out of 5728 companies in our database      https   t co Cr9bZVzF6Z  AKAM        https   t co ieTruwLxWJ
Akamai Technologies  Inc current quarter revenue growth YoY of 7 00  ranked 2639 out of 5728 companies in our database      https   t co Cr9bZVzF6Z  AKAM        https   t co tDmgbJqofE
Call Put Highest   Volume Changes   Calls   COOP   TWTR   MFA   KTOS   ATER Puts   TWTR   RAD   AKAM   FSR   JMIA
 AKAM  110 00 Aug 19 2

Allegion plc current quarter revenue growth YoY of  2 50  ranked 3322 out of 5728 companies in our database      https   t co xTg89t95fD  ALLE        https   t co AS4Q5U9Z5M
Rose Above Previous Day s High today   XDEC  HTPA  DTSS  QED  ATY  FSMD  CVY  SYNL  CM  GSID  BLKC  ALLE  CNCR  ARTE  CLSM  IBDN  TCX  BWAY  IMV  GSEE     https   t co TBYRpvrFqi
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff https   t co Wkenzh7ovw
 ALLE   Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff  https   t co Yn1ajAW4W8  45 seconds delayed 
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff   https   t co Ny0tB8qI4E
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff https   t co cfai

...

New 52 Week High today   HAL  EIX  XEL  HSY  CLF  KMI  CMS  ETR  CNP  CEIX  TDY  FE  COPX  SLCA  OGS  PBR  ZETA  RRC  IMO  NRP     https   t co 4Iw0QeR3p8
If you invest  1000 in  CRMT today at  82 16 per share  and the stock prices rebound back to pre pandemic prices at  117 97  your investment would be worth  1435 86  That s 44  growth and  435 86 profit     See more  https   t co 2pV8Epn69z    NRK  TEX  SGRP  LTBR  EIX https   t co 90YYzZRW4N
Our Unusual Options Platform Alerted   EIX with our scanner on  65 CALL Expiring  07 15 2022 worth 30K    Get Ahead Of The Move with SweepCast  amp  Learn more about it on our profile today   https   t co wJsMPDd9RE
Nice print for  EIX Size  233600 Price  70 8 Amount   16 538 880 00 Time  1830 See more  https   t co kiRZSq7W5C Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
There are 40 companies in the S amp P 500  amp  NASDAQ 100 that 

New 52 Week High today   HAL  EIX  XEL  HSY  CLF  KMI  CMS  ETR  CNP  CEIX  TDY  FE  COPX  SLCA  OGS  PBR  ZETA  RRC  IMO  NRP     https   t co 4Iw0QeR3p8
Entergy Corp  Shares Approach 52 Week High   Market Mover https   t co tOja2LpgzV  ETR  stockmarket  kwhenfinance
Nice print for  ETR Size  268596 Price  119 41 Amount   32 073 048 36 Time  1830 See more  https   t co D0FMKJC5bD Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
Entergy Corporation current quarter revenue growth YoY of 14 90  ranked 2048 out of 5728 companies in our database      https   t co 2MjoZjihNC  ETR
There are 40 companies in the S amp P 500  amp  NASDAQ 100 that that are Overbought based on RSI today   ADP   AEE   AEP   AES   AMGN   BMY   CCI   CMS   CNP   COST   DTE   DUK   ED   EIX   ENPH   ETR   EVRG   EXC   FAST   FE   INCY   IRM   JKHY   MCK   MRK   NLSN   PAYX   PNW   SBAC 
Monday s Top Flow in  u

Rose Above Upper Bollinger Band today   ESS     https   t co AyIukUGmib
 ESS  NYSE  Stock Rating Changed to Buy  Essex Property Trust Inc  gt  gt  https   t co zLCoT3mMO4
 Scan results    Gateio   15m   Top 5 bullish trend on  BTC pair 1   BNB 2   FIL 3   SOLO 4   SSV 5   WBTC  Top 5 bullish trend on  USDT pair 1   SASHIMI 2   RMRK 3   ESS 4   SMTY 5   GOFX  All  Gateio results  https   t co MZdL0PuAQU Not buy signals
Market went bonkers for lithium yesterday day it like puked itself  ESS  MRR
Li shares are coming from morning sell down   AGY   ESS  BNR  QXR
Bought the  ESS dip  Mmm  that s delicious  https   t co IXdqywYFzv
Rose Above Previous Day s High today   FIN  MOH  PLY  NVX  ELO  COL  A1G  ESS  TPG  EGR  ASIA  FEX  GSM  CAU  ARX  XTC     https   t co Sr2t6babZ6
 LPI   1   breached  Overdue and expected a lot sooner but better late then never   Now we play catch up to  LKE   For Hard Rock with proven resources have a look into  ESS  I feel it s still good value here with further

Nice print for  EXC Size  158800 Price  48 025 Amount   7 626 370 00 Time  1504 See more  https   t co RBuIfpY6po Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
Midday  QQQ is trading down from a pivot high resistance and its 200 day average  Stocks that were strong yesterday are being dumped today  Stocks trading higher   VRTX  SGEN  ADP  AEP  XEL  AZN  EXC  REGN  PAYX https   t co U8Frbv8BL6
Nice print for  EXC Size  1072300 Price  47 93 Amount   51 395 339 00 Time  1245 See more  https   t co RBuIfpY6po Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
Mid day highest Standard Deviation gain compared to the previous 20 days  Min  5  Vol 500K  Call Vol 500K    SST  THCA  MRK  BMY  ABBV  NRGV  GSK  DUK  SO  NNOX  UNH  EXC  AMT  VRTX  JNJ  BAX  NEE  ADP  KR  GIS  AZN  D 

 FFIV https   t co xetZR5EAzm F5 Inc  The current setup has detected this company s stock price has an undetermined short term setup and its stock price is in line with its long term fundamentals  Investing  news  trade
 FFIV https   t co xetZR5EAzm F5 Inc  Our predictive algorithm has predicted the market value of this company has an undetermined short term setup with a flat long term setup  Finance  StockMarket  trade
Insider Activity seen in   FFIV  WHITE ANA MARIA  EVP and Chief H  Sale  Total Shares   169  Transaction Price   209 94  Transaction Value   35 480  Last Quote   214 24 https   t co e6ocCf0ZgV
Insider Activity seen in   FFIV  Locoh Donou Francois  President  CEO  Sale  Total Shares   1 042  Transaction Price   210 30  Transaction Value   219 133  Last Quote   214 24 https   t co 7jyExbPhF9
 FFIV   SEC Form 4  Locoh Donou Francois sold  219 133 worth of shares  1 042 units at  210 30  as part of a pre agreed trading plan  decreasing direct ownership by 1  to 93 435 units

In [8]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)

sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)

for row in alltweets:
    blob = TextBlob(row[3])
    if blob.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive"])
    elif blob.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative"])     
    elif blob.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral"])    
        
file.close()
sentiment_file.close()